In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'1.14.0'

In [2]:
def _l2normalize(v, eps=1e-12):
    return v / (tf.reduce_sum(v ** 2) ** 0.5 + eps)

def spectral_norm(weights, name, num_iters=1, update_collection=None, with_sigma=False):
    w_shape = weights.shape.as_list()
    w_mat = tf.reshape(weights, [-1, w_shape[-1]])  # [-1, output_channel]
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        u = tf.get_variable('u', [1, w_shape[-1]],
                          initializer=tf.truncated_normal_initializer(),
                          trainable=False)
        print(u.name)
    u_ = u
    for _ in range(num_iters):
        v_ = _l2normalize(tf.matmul(u_, w_mat, transpose_b=True))
        u_ = _l2normalize(tf.matmul(v_, w_mat))

    sigma = tf.squeeze(tf.matmul(tf.matmul(v_, w_mat), u_, transpose_b=True))
    w_mat /= sigma
    if update_collection is None:
        with tf.control_dependencies([u.assign(u_)]):
            w_bar = tf.reshape(w_mat, w_shape)
            print('u is updated')
    else:
        w_bar = tf.reshape(w_mat, w_shape)
        print('u is NOT updated')
        if update_collection != 'NO_OPS':
            tf.add_to_collection(update_collection, u.assign(u_))
    if with_sigma:
        return w_bar, sigma
    else:
        return w_bar

In [3]:
class SNDense(tf.keras.layers.Layer):
    def __init__(self, units, 
                 kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.02), 
                 bias_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.02), 
                 update_collection=None, **kwargs):
        super(SNDense, self).__init__(**kwargs)
        self.units = units
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.update_collection = update_collection
        
    
    def build(self, input_shape):
        '''
        input_shape = (batch_size, ..., input_dim)
        '''
        self.input_dim = int(input_shape[-1])
        with tf.variable_scope(self.name, reuse=tf.AUTO_REUSE):
            self.kernel = tf.get_variable("kernel", 
                                          shape=(self.input_dim, self.units), 
                                          initializer=self.kernel_initializer)
            self.bias = tf.get_variable("bias", shape=(self.units,), initializer=self.bias_initializer)
        super(SNDense, self).build(input_shape)  # Be sure to call this at the end
    
    
    def call(self, inputs):
        x = tf.matmul(inputs, spectral_norm(self.kernel, name=self.name, update_collection=self.update_collection))
        x = tf.nn.bias_add(x, self.bias)
        return x

W1126 17:32:36.642154 18244 deprecation.py:506] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\keras\initializers.py:94: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
class Block(tf.keras.layers.Layer):
    def __init__(self, update_collection=None, **kwargs):
        super(Block, self).__init__(**kwargs)
        self.update_collection = update_collection
        
    
    def build(self, input_shape):
        self.fc1 = SNDense(20, update_collection=self.update_collection, name='fc1')
        self.fc2 = SNDense(10, update_collection=self.update_collection, name='fc2')
        with tf.variable_scope(self.name, reuse=tf.AUTO_REUSE):
            self.bias = tf.get_variable("bias", shape=(10,), initializer=tf.constant_initializer(0))
        super(Block, self).build(input_shape)  # Be sure to call this at the end
    
    
    def call(self, inputs):
        with tf.variable_scope(self.name, reuse=tf.AUTO_REUSE):
            x = self.fc1(inputs)
            x = self.fc2(x)
            x = tf.nn.bias_add(x, self.bias)
            return x

In [5]:
def make_model(update_collection):
    x = tf.keras.layers.Input(shape=(10,))
    y = Block(name='block1', update_collection=update_collection)(x)
    y = Block(name='block2', update_collection=update_collection)(x)
    return tf.keras.models.Model(x, y)

In [6]:
model = make_model(update_collection=None)

W1126 17:32:37.275648 18244 deprecation_wrapper.py:119] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\autograph\converters\directives.py:117: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1126 17:32:37.275648 18244 deprecation_wrapper.py:119] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\autograph\converters\directives.py:117: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W1126 17:32:37.310119 18244 ag_logging.py:145] Entity <bound method Block.call of <__main__.Block object at 0x000001C964751128>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Block.call of <__main__.Block object at 0x000001C964751128>>: AssertionError: Bad argument number for Name: 3, e

block1/fc1/u:0
u is updated


W1126 17:32:37.504435 18244 ag_logging.py:145] Entity <bound method SNDense.call of <__main__.SNDense object at 0x000001C971BDB630>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method SNDense.call of <__main__.SNDense object at 0x000001C971BDB630>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1126 17:32:37.622218 18244 ag_logging.py:145] Entity <bound method Block.call of <__main__.Block object at 0x000001C96E5CFC18>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Block.call of <__main__.Block object at 0x000001C96E5CFC18>>: AssertionError: Bad argument number for Name: 3, expecting 

block1/fc2/u:0
u is updated


W1126 17:32:37.723226 18244 ag_logging.py:145] Entity <bound method SNDense.call of <__main__.SNDense object at 0x000001C971BE8320>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method SNDense.call of <__main__.SNDense object at 0x000001C971BE8320>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1126 17:32:37.837105 18244 ag_logging.py:145] Entity <bound method SNDense.call of <__main__.SNDense object at 0x000001C971BE85F8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method SNDense.call of <__main__.SNDense object at 0x000001C971BE85F8>>: AssertionError: Bad argument number for Name: 3, ex

block2/fc1/u:0
u is updated
block2/fc2/u:0
u is updated


In [7]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[<tf.Variable 'block1/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'block1/fc1/kernel:0' shape=(10, 20) dtype=float32_ref>,
 <tf.Variable 'block1/fc1/bias:0' shape=(20,) dtype=float32_ref>,
 <tf.Variable 'block1/fc1/u:0' shape=(1, 20) dtype=float32_ref>,
 <tf.Variable 'block1/fc2/kernel:0' shape=(20, 10) dtype=float32_ref>,
 <tf.Variable 'block1/fc2/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'block1/fc2/u:0' shape=(1, 10) dtype=float32_ref>,
 <tf.Variable 'block2/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'block2/fc1/kernel:0' shape=(10, 20) dtype=float32_ref>,
 <tf.Variable 'block2/fc1/bias:0' shape=(20,) dtype=float32_ref>,
 <tf.Variable 'block2/fc1/u:0' shape=(1, 20) dtype=float32_ref>,
 <tf.Variable 'block2/fc2/kernel:0' shape=(20, 10) dtype=float32_ref>,
 <tf.Variable 'block2/fc2/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'block2/fc2/u:0' shape=(1, 10) dtype=float32_ref>]

In [8]:
model_no_op = make_model(update_collection='NO_OPS')

W1126 17:33:13.824445 18244 ag_logging.py:145] Entity <bound method Block.call of <__main__.Block object at 0x000001C971EC1B00>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Block.call of <__main__.Block object at 0x000001C971EC1B00>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1126 17:33:13.870006 18244 ag_logging.py:145] Entity <bound method SNDense.call of <__main__.SNDense object at 0x000001C971ED80F0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method SNDense.call of <__main__.SNDense object at 0x000001C971ED80F0>>: AssertionError: Bad argument number for Name: 3, expecting 

block1/fc1/u:0
u is NOT updated
block1/fc2/u:0
u is NOT updated


W1126 17:33:14.043545 18244 ag_logging.py:145] Entity <bound method Block.call of <__main__.Block object at 0x000001C971EC1CF8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Block.call of <__main__.Block object at 0x000001C971EC1CF8>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1126 17:33:14.113294 18244 ag_logging.py:145] Entity <bound method SNDense.call of <__main__.SNDense object at 0x000001C971E27AC8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method SNDense.call of <__main__.SNDense object at 0x000001C971E27AC8>>: AssertionError: Bad argument number for Name: 3, expecting 

block2/fc1/u:0
u is NOT updated


W1126 17:33:14.269375 18244 ag_logging.py:145] Entity <bound method SNDense.call of <__main__.SNDense object at 0x000001C971E5ED68>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method SNDense.call of <__main__.SNDense object at 0x000001C971E5ED68>>: AssertionError: Bad argument number for Name: 3, expecting 4


block2/fc2/u:0
u is NOT updated


In [9]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[<tf.Variable 'block1/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'block1/fc1/kernel:0' shape=(10, 20) dtype=float32_ref>,
 <tf.Variable 'block1/fc1/bias:0' shape=(20,) dtype=float32_ref>,
 <tf.Variable 'block1/fc1/u:0' shape=(1, 20) dtype=float32_ref>,
 <tf.Variable 'block1/fc2/kernel:0' shape=(20, 10) dtype=float32_ref>,
 <tf.Variable 'block1/fc2/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'block1/fc2/u:0' shape=(1, 10) dtype=float32_ref>,
 <tf.Variable 'block2/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'block2/fc1/kernel:0' shape=(10, 20) dtype=float32_ref>,
 <tf.Variable 'block2/fc1/bias:0' shape=(20,) dtype=float32_ref>,
 <tf.Variable 'block2/fc1/u:0' shape=(1, 20) dtype=float32_ref>,
 <tf.Variable 'block2/fc2/kernel:0' shape=(20, 10) dtype=float32_ref>,
 <tf.Variable 'block2/fc2/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'block2/fc2/u:0' shape=(1, 10) dtype=float32_ref>]

In [ ]:
output_no_op = model_no_op(model_no_op.input)
output = model(model.input)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print('Initial value: {}'.format(sess.run('fc1/u:0')))
    
    sess.run(output_no_op, feed_dict={model_no_op.input: 2*np.ones((5,10))})
    print('After run NO_OPS : {}'.format(sess.run('fc1/u:0')))
    
    sess.run(output, feed_dict={model.input: 2*np.ones((5,10))})
    print('After run None : {}'.format(sess.run('fc1/u:0')))
    
    sess.run(output_no_op, feed_dict={model_no_op.input: 2*np.ones((5,10))})
    print('After run NO_OPS : {}'.format(sess.run('fc1/u:0')))
    
    sess.run(output, feed_dict={model.input: np.ones((5,10))})
    print('After run None : {}'.format(sess.run('fc1/u:0')))